In [1]:
import boto3
import os
import pandas as pd
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.context import SparkContext as sc
import csv

os.environ["PYSPARK_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"

In [2]:
accessKeyId='accessKeyId'
secretAccessKey='secretAccessKey'

s3 = boto3.client("s3", 
                  region_name='us-east-1', 
                  aws_access_key_id=accessKeyId, 
                  aws_secret_access_key=secretAccessKey)

In [4]:
files=s3.list_objects(Bucket="yunus-input")["Contents"]
len(files)

9

In [5]:
cwd=os.getcwd()
cwd=cwd+"/deneme/"
print (cwd)

/home/yunus/deneme/


In [6]:
for file in files:
    s3.download_file(Bucket="yunus-input",
    Key=file["Key"],
    Filename=cwd+"download_"+file["Key"])

In [7]:
spark= pyspark.sql.SparkSession.builder.getOrCreate()

22/03/25 00:48:39 WARN Utils: Your hostname, yunus resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface enp34s0)
22/03/25 00:48:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 00:48:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/25 00:48:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
path ='/home/yunus/deneme/download_olist_order_items_dataset.csv'

data = spark.read.csv(path,
                     sep=',',
                     header=True)
data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- freight_value: string (nullable = true)



In [9]:
path2 ='/home/yunus/deneme/download_olist_orders_dataset.csv'

data2 = spark.read.csv(path2,
                     sep=',',
                     header=True)
data2.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [10]:
data.createOrReplaceTempView('data')
data2.createOrReplaceTempView('data2')
deneme= spark.sql(r"""Select * from data""").show()
#deneme.show()
deneme2= spark.sql('Select * from data2')
deneme2.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.00|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.90|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [11]:
result=spark.sql('select b.is_late,count(*) from(select a.order_id,a.seller_id,a.y,a.x,\
                        case when datediff(TO_DATE(CAST(UNIX_TIMESTAMP(a.y, "yyyy/MM/dd") AS TIMESTAMP)),TO_DATE(CAST(UNIX_TIMESTAMP(a.x, "yyyy/MM/dd") AS TIMESTAMP)))>0 then 0 else 1 end is_late\
                    from (select a.order_id,b.seller_id,\
                                 to_date(a.order_delivered_carrier_date) as x,\
                                 to_date(a.order_estimated_delivery_date) as y \
                            from data2 a left join data b on a.order_id=b.order_id \
                                where  a.order_delivered_carrier_date is not null \
                                and a.order_estimated_delivery_date is not null) a)b group by b.is_late').show()

+-------+--------+
|is_late|count(1)|
+-------+--------+
|      1|     544|
|      0|  110913|
+-------+--------+



In [12]:
result=spark.sql('select b.seller_id,b.is_late from(select a.order_id,a.seller_id,a.y,a.x,\
                        case when datediff(TO_DATE(CAST(UNIX_TIMESTAMP(a.y, "yyyy/MM/dd") AS TIMESTAMP)),TO_DATE(CAST(UNIX_TIMESTAMP(a.x, "yyyy/MM/dd") AS TIMESTAMP)))>0 then 0 else 1 end is_late\
                    from (select a.order_id,b.seller_id,\
                                 to_date(a.order_delivered_carrier_date) as x,\
                                 to_date(a.order_estimated_delivery_date) as y \
                            from data2 a left join data b on a.order_id=b.order_id \
                                where  a.order_delivered_carrier_date is not null \
                                and a.order_estimated_delivery_date is not null) a) b where b.is_late=1')

In [13]:
result_data = result.toPandas()
result_data.head(10)

,seller_id,is_late
0,633ecdf879b94b5337cca303328e4a25,1
1,fd386aa7bed2af3c7035c65506c9b4a3,1
2,cac4c8e7b1ca6252d8f20b2fc1a2e4af,1
3,4b5f66b7adcf57f1ecc0d3c07dd6b177,1
4,b19f3ca2ea475913750f25a5c37c8d8f,1
5,44f091b5abab8018f682fce8124b85e5,1
6,fa1c13f2614d7b5c4749cbc52fecda94,1
7,c70c1b0d8ca86052f45a432a38b73958,1
8,89bc797e2e29667aa035d4368f9b7f92,1
9,8d956fec2e4337affcb520f56fd8cbfd,1


In [14]:
result_data.to_csv('is_late_table')